# 下载基金排行
需要下载http://fund.eastmoney.com/data/fundranking.html#tgp;c0;r;s1nzf;pn50;ddesc;qsd20210926;qed20220926;qdii;zq;gg;gzbd;gzfs;bbzt;sfbb
上的基金排行到fundrank.csv中

如何将网站上的信息down下来存放在csv中，首先需要了解爬虫，爬虫是如何从网站上爬下来数据，并将数据存放在csv中的。
需要了解下爬虫的知识点
## 编写爬虫的流程
爬虫程序与其他程序不同，它的的思维逻辑一般都是相似的， 所以无需我们在逻辑方面花费大量的时间。下面对 Python 编写爬虫程序的流程做简单地说明：
先由 urllib 模块的 request 方法打开 URL 得到网页 HTML 对象。
使用浏览器打开网页源代码分析网页结构以及元素节点。
通过 Beautiful Soup 或则正则表达式提取数据。
存储数据到本地磁盘或数据库。
## 前端知识
爬虫程序之所以可以抓取数据，是因为爬虫能够对网页进行分析，并在网页中提取出想要的数据。在学习 Python 爬虫模块前，我们有必要先熟悉网页的基本结构，这是编写爬虫程序的必备知识。
如果您熟悉前端语言，那么您可以轻松地掌握本节知识。
网页一般由三部分组成，分别是 HTML（超文本标记语言）、CSS（层叠样式表）和 JavaScript（简称“JS”动态脚本语言），它们三者在网页中分别承担着不同的任务。
* HTML 负责定义网页的内容
* CSS 负责描述网页的布局
* JavaScript 负责网页的行为
由于前端内容太多，爬数据其实爬的是网页内容，所以学习下HTML先

In [ ]:
 <!DOCTYPE html> <!--声明为 HTML5 文档 -->
<html><!--是网页的根元素 -->
<head><!--元素包含了文档的元（meta）数据，如 <meta charset="utf-8"> 定义网页编码格式为 utf-8。 -->
<meta charset="utf-8">
<title>编程帮</title><!--元素描述了文档的标题 -->
</head>
<body><!--表示用户可见的内容 -->
<a href="www.biancheng.net">点击访问</a><!--表示超链接 -->
<h1>编程帮www.biancheng.net</h1><!--标题 -->
<h2>Python爬虫</h2><!--标题 -->
<div><!--表示框架 -->
<p>认识网页结构</p><!--段落 -->
<ul><!--定义无序列表 -->
<li>HTML</li><!--表示列表项 -->
<li>CSS</li><!--表示列表项 -->
</ul>
</div>
</body>
</html>

## 最初的爬虫
一个完整的爬虫由request请求模块来打开网页并返回response
并需要提取响应内容，我们需要读取响应内容并打印内容，例子如下

In [7]:
#导包,发起请求使用urllib库的request请求模块
import urllib.request
# urlopen()向URL发请求,返回响应对象,注意url必须完整
import re
url='http://fund.eastmoney.com/data/fundranking.html#tgp;c0;r;sqjzf;pn10000;ddesc;qsd20210927;qed20220927;qdii;zq;gg;gzbd;gzfs;bbzt;sfbb'
headers = {'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36'}
req = urllib.request.Request(url=url,headers=headers)
res = urllib.request.urlopen(req)
# res=urllib.request.urlopen(req)#先由 urllib 模块的 request 方法打开 URL 得到网页 HTML 对象。
# print(res)#打印返回结果
#提取响应内容
html = res.read().decode('utf-8')
#打印响应内容
print(html)#html篇幅过长
re_bds = '<a href="(.*?)" title="(.*?)">(.*?)</a>'
pttn=re.compile(re_bds)
r_list = re.findall(pttn,html)
print(r_list)




<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
    <title>开放式基金排行 _ 天天基金网</title>
    <meta name="keywords" content="基金排行,开放式基金排行,创新基金排行,货币基金排行,涨幅排行,基金排行查询,涨幅分布,自定义基金排行,股票型基金,混合型基金,债券型基金,指数型基金,保本型基金,QDII,LOF,按基金公司筛选" />
    <meta name="description" content="天天基金网每日及时更新开放式基金收益率排行。" />
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <meta name="mobile-agent" content="format=html5; url=https://m.1234567.com.cn/?page=jjph&tab=qb" />
    <meta http-equiv="Content-Language" content="zh-CN" />
    <meta http-equiv="Cache-Control" content="no-cache" />
    <meta http-equiv="Expires" content="-1" />

    <script type="text/javascript" src="//j5.dfcfw.com/js/data/hq_fund_20140425093502.js?v=20221003210431"></script>
    <script type="text/javascript" src="//fund.eastmoney.com/js/fundcode_search.js?v=20221003210431"></script>
    <script typ

### 常用方法
在本节您认识了第一个爬虫库 urllib，下面关于 urllib 做简单总结。
1) urlopen()
表示向网站发起请求并获取响应对象，如下所示：
```python
urllib.request.urlopen(url,timeout)
```
urlopen() 有两个参数，说明如下：
url：表示要爬取数据的 url 地址。
timeout：设置等待超时时间，指定时间内未得到响应则抛出超时异常。
2) Request()
该方法用于创建请求对象、包装请求头，比如重构 User-Agent（即用户代理，指用户使用的浏览器）使程序更像人类的请求，而非机器。重构 User-Agent 是爬虫和反爬虫斗争的第一步。在下一节会做详细介绍。
```python
urllib.request.Request(url,headers)
```
参数说明如下：
url：请求的URL地址。
headers：重构请求头。

3) html响应对象方法
```python
bytes = response.read() # read()返回结果为 bytes 数据类型
string = response.read().decode() # decode()将字节串转换为 string 类型
url = response.geturl() # 返回响应对象的URL地址
code = response.getcode() # 返回请求时的HTTP响应码
```
4) 编码解码操作
```python
#字符串转换为字节码
string.encode("utf-8") 
#字节码转换为字符串
bytes.decode("utf-8") 
```

In [51]:
from urllib import request
import re
import time
import random
import csv
from ua_info import ua_list
# 定义一个爬虫类
class MaoyanSpider(object): 
    # 初始化
    # 定义初始页面url
    def __init__(self):
        self.url = 'https://www.maoyan.com/board/4?offset={}'
    
    # 请求函数
    def get_html(self,url):
        headers = {'User-Agent':random.choice(ua_list)}
        req = request.Request(url=url,headers=headers)
        res = request.urlopen(req)
        html = res.read().decode()
        # 直接调用解析函数
        self.parse_html(html)
    
    # 解析函数
    def parse_html(self,html):
        # 正则表达式
        # '<div class="movie-item-info"><a href="/films/.*" title="(.*?)" data-act="boarditem-click" data-val="{movieId:.*?}">(.*?)</a><p class="name"></p><p class="star">(.*?)</p><p class="releasetime">(.*?)</p>'
        # '<a href="/films/1200486" title="我不是药神" data-act="boarditem-click" data-val="{movieId:1200486}">我不是药神</a>'
        re_bds = '<div class="movie-item-info"><a href="/films/.*" title="(.*?)" data-act="boarditem-click" data-val="{movieId:.*?}">(.*?)</a><p class="name"></p><p class="star">(.*?)</p><p class="releasetime">(.*?)</p>'
        # 生成正则表达式对象
        pattern = re.compile(re_bds,re.S)
        # r_list: [('我不是药神','徐峥,周一围,王传君','2018-07-05'),...] 列表元组
        r_list = pattern.findall(html)
        self.save_html(r_list)
    # 保存数据函数，使用python内置csv模块
    def save_html(self,r_list):
        #生成文件对象  
        with open('maoyan.csv','a',newline='',encoding="utf-8") as f:
            #生成csv操作对象
            writer = csv.writer(f)
            #整理数据
            for r in r_list:
                name = r[0].strip()
                star = r[1].strip()[3:]
                # 上映时间：2018-07-05
                # 切片截取时间
                time = r[2].strip()[5:15]
                L = [name,star,time]
                # 写入csv文件
                writer.writerow(L)
                print(name,time,star)
    # 主函数
    def run(self):
        #抓取第一页数据
        for offset in range(0,11,10):
            url = self.url.format(offset)
            self.get_html(url)
            #生成1-2之间的浮点数
            time.sleep(random.uniform(1,2))
# 以脚本方式启动
if __name__ == '__main__':
    #捕捉异常错误
    try:
        spider = MaoyanSpider()
        spider.run()
    except Exception as e:
        print("错误:",e)

错误: HTTP Error 302: The HTTP server returned a redirect error that would lead to an infinite loop.
The last 30x error message was:
Found


In [6]:
#导入parse模块
from urllib import parse
#构建查询字符串字典
query_string = {
'wd' : '天天'
}
#调用parse模块的urlencode()进行编码
result = parse.urlencode(query_string)
#使用format函数格式化字符串，拼接url地址
url = 'http://www.baidu.com/s?{}'.format(result)
print(url)

http://www.baidu.com/s?wd=%E5%A4%A9%E5%A4%A9


In [ ]:
from urllib import request,parse
# 1.拼url地址
url = 'http://www.baidu.com/s?wd={}'
word = input('请输入搜索内容:')
params = parse.quote(word)
full_url = url.format(params)
# 2.发请求保存到本地
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0) Gecko/20100101 Firefox/6.0'}
req = request.Request(url=full_url,headers=headers)
res = request.urlopen(req)
html = res.read().decode('utf-8')
# 3.保存文件至当前目录
filename = word + '.html'
with open(filename,'w',encoding='utf-8') as f:
    f.write(html)

re模块常用方法
--------

#### 1) re.compile()

该方法用来生成正则表达式对象，其语法格式如下：
```python
regex=re.compile(pattern,flags=0)
```
参数说明：

-   pattern：正则表达式对象。
-   flags：代表功能标志位，扩展正则表达式的匹配。

#### 2) re.findall()

根据正则表达式匹配目标字符串内容。
```python
re.findall(pattern,string,flags=0)
```
该函数的返回值是匹配到的内容列表，如果正则表达式有子组，则只能获取到子组对应的内容。参数说明如下：

-   pattern：正则表达式对象。
-   string：目标字符串
-   flags：代表功能标志位，扩展正则表达式的匹配。

#### 3) regex.findall()

该函数根据正则表达式对象匹配目标字符串内容。其语法格式如下：
```python
regex.findall(string,pos,endpos)
```
参数说明：

-   string 目标字符串。
-   pos 截取目标字符串的开始匹配位置。
-   endpos 截取目标字符串的结束匹配位置。

#### 4) re.split()

该函数使用正则表达式匹配内容，切割目标字符串。返回值是切割后的内容列表。参数说明：
```python
re.split(pattern,string,flags = 0)
```
参数说明：

-   pattern：正则表达式。
-   string：目标字符串。
-   flags：功能标志位,扩展正则表达式的匹配。

5) re.sub\
该函数使用一个字符串替换正则表达式匹配到的内容。返回值是替换后的字符串。其语法格式如下：
```python
re.sub(pattern,replace,string,max,flags = 0)
```
其参数说明：

-   pattern：正则表达式。
-   replace：替换的字符串。
-   string：目标字符串。
-   max：最多替换几处，默认替换全部，
-   flags：功能标志位,扩展正则表达式的匹配。

#### 5) re.search()

匹配目标字符串第一个符合的内容，返回值为匹配的对象。语法格式如下：

```python
re.search(pattern,string,flags=0)
```

参数说明：

-   pattern：正则表达式
-   string：目标字符串

flags功能标志位
----------

功能标志位的作用是扩展正则表达的匹配功能。常用的 flag 如下所示：

flag功能标志位
| 缩写元字符 | 说明 |
| A | 元字符只能匹配 ASCII码。 |
| I | 匹配忽略字母大小写。 |
| S | 使得`.`元字符可以匹配换行符。 |
| M | 使 ^ $ 可以匹配每一行的开头和结尾位置。 |\
注意：可以同时使用福多个功能标志位，比如 flags=re.I|re.S。


In [31]:
import re
import regex
str ='violation_ad_spend_amt_28d_total DECIMAL(18,2) COMMENT \'违规广告近28日总消耗 - 大盘分子\''
pttn = re.compile(r'\d+')
str2 = re.findall(pttn,str)#找到所有匹配元素
print(str)
print(str2)
str3= re.split(pttn,str,flags = 0)#按照正则进行分割
print(str3)
str4=re.sub(pttn,'fuck',str,2)
print(str4)
str5 = re.search(pttn,str,flags=0)#返回第一个匹配所在的位置
print(str5)

violation_ad_spend_amt_28d_total DECIMAL(18,2) COMMENT '违规广告近28日总消耗 - 大盘分子'
['28', '18', '2', '28']
['violation_ad_spend_amt_', 'd_total DECIMAL(', ',', ") COMMENT '违规广告近", "日总消耗 - 大盘分子'"]
violation_ad_spend_amt_fuckd_total DECIMAL(fuck,2) COMMENT '违规广告近28日总消耗 - 大盘分子'
<re.Match object; span=(23, 25), match='28'>


## csv写入

In [32]:
import csv
with open('names.csv', 'w', newline='') as csvfile:
    #构建字段名称，也就是key
    fieldnames = ['first_name', 'last_name']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    # 写入字段名，当做表头
    writer.writeheader()
    # 多行写入
    writer.writerows([{'first_name': 'Baked', 'last_name': 'Beans'},{'first_name': 'Lovely', 'last_name': 'Spam'}])
    # 单行写入
    writer.writerow({'first_name': 'Wonderful', 'last_name': 'Spam'})